In [1]:
import tensorflow as tf
import numpy as np
#import mnist_handler
import os
import pandas as pd
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"]="3"; 
INPUT_DIMENSION = 180
NUM_CLASSES = 3

def process_data(fname, NUM_ATTR, NUM_CLASSES):
    I = np.eye(NUM_CLASSES)
    with open(fname) as file:
        xx = file.readlines()
        #print(len(xx))
        data = np.zeros([len(xx), NUM_ATTR])
        #print(data.shape)
        label = np.zeros(len(xx), dtype=int)
        label_onehot = []
        for i in range(len(xx)):
            tmp = xx[i].split(' ')
            #print(tmp)
           # print(len(tmp))
            label[i] = int(tmp[0])
            if label[i] == 2:
                label[i] = 1
            #print(label[i])
            label_onehot.append(I[label[i]])
            for j in range(1,len(tmp)-1):
                #print(tmp[j])
                if tmp[j] == '':
                    continue
                #print(i)
                #print(tmp[j].split(':'))
                position = tmp[j].split(':')[0]
                position = int(position)
                value = 1*float(tmp[j].split(':')[1])
                
                data[i][position-1] = value 
    return data, label, np.array(label_onehot, dtype=int)

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

tr_file = 'UCI dataset/protein'
ts_file = 'UCI dataset/protein.t'

In [3]:
tr_data, tr_label, tr_onehot = process_data(tr_file, NUM_ATTR=357, NUM_CLASSES=2)
ts_data, ts_label, ts_onehot = process_data(ts_file, NUM_ATTR=357, NUM_CLASSES=2)


In [4]:
x_train = tr_data*0.01
y_train = tr_label
x_test = ts_data*0.01
y_test = ts_label
x_train
x_test.shape

(6621, 357)

np.savetxt("x_train.csv", x_train, delimiter=",")
np.savetxt("y_train.csv", y_train, delimiter=",")
np.savetxt("x_test.csv", x_test, delimiter=",")
np.savetxt("y_test.csv", y_test, delimiter=",")

In [5]:

border = 621
x_train_init = x_train[:border]
y_train_init = y_train[:border]
x_train_seq = x_train[border:]
y_train_seq = y_train[border:]

import time
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(0.55)
time_sum = 0
t1 = time.time()
pca.fit(x_train)
t2 = time.time()
time_sum+= t2-t1
train_img = pca.transform(x_train)
test_img  = pca.transform(x_test)
x_train.shape,train_img.shape

In [7]:
x_train.shape,y_train.shape
INPUT_DIMENSION = 357
NUM_CLASSES = 2

border = 146
x_train_init = train_img[:border]
y_train_init = y_train[:border]
x_train_seq = train_img[border:]
y_train_seq = y_train[border:]

In [8]:
tf.reset_default_graph() # clear all the tensors
sess = tf.Session()


"""Placeholders"""
X_ = tf.placeholder(tf.float32, [None, INPUT_DIMENSION])
#X_ = tf.reshape(X, [-1, INPUT_DIMENSION]) # Flatten X: [N,D]
Y = tf.placeholder(tf.int64, [None]) # labels
Y_ = tf.one_hot(indices=Y, depth=NUM_CLASSES) # one_hot labels: [N,M]

"""Some constants"""
D = INPUT_DIMENSION
M = NUM_CLASSES # Number of outputs
C = tf.constant(2.0**(-3))

"""Weights"""
alpha_1 = tf.get_variable('alpha_1',shape=[D, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False) # 1st subnetwork
alpha_2 = tf.get_variable('alpha_2',shape=[D, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False) # 2st subnetwork
alpha_3 = tf.get_variable('alpha_3',shape=[D, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
beta_1 = tf.get_variable('beta_1',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
beta_2 = tf.get_variable('beta_2',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
beta_3 = tf.get_variable('beta_3',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)

In [9]:
k = tf.get_variable('k',shape=[D, D],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
m = tf.get_variable('m',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)

In [10]:
"""Utility functions"""
def mul(A, B):
    return tf.matmul(A, B)

def inv(A):
    return tf.matrix_inverse(A)

def t(A):
    return tf.transpose(A)

def sin(A):
    return tf.math.sin(A)

def asin(A):
    return tf.math.asin(A)

def sqrt(A):
    return tf.sqrt(A)

def sqr(A):
    return tf.math.pow(A, 2)

def pseudo_inv(A, I, C):
    C_I = I/C
    return mul(t(A), inv(C_I + mul(A, t(A))))

def h(A):
    '''activation function'''
    return sin(A)

def h_(A):
    '''inverse activation function'''
    return asin(A)

def u(A):
    '''normalize the input to (0,1]'''
    return tf.math.sigmoid(A) # sigmoid
    
def u_(A):
    '''the inverse of u'''
    ONE = tf.constant(1.0)
    return -(tf.math.log(ONE/A - ONE)) # the inverse of sigmoid
    
def subnet_output(alpha, beta, A):
    return t(mul(beta, h(mul(t(alpha), t(A))))) 

# Initial Training

In [11]:
"""Initial Training Graph"""
# D: input dimension
# N: number of input samples
# M: number of classes (number of outputs)
'''some pre-computations'''
X_init = t(X_) # [D,N]
Y_init = t(Y_) # [M,N]
N_init = D # number of dimensions
I_DxD = tf.eye(N_init, dtype=tf.float32) # [D,D]
I_MxM = tf.eye(M, dtype=tf.float32) # [M,M]
C_I = I_DxD/C
H_I = I_MxM/C

add = C_I + mul(X_init, t(X_init))
k = tf.assign(k,add)
X_inv_init = pseudo_inv(X_init, I_DxD, C) # [N,D]

'''1st subnet'''
alpha_1_init_calculated = t(mul(h_(Y_init), X_inv_init)) # ([M,N]x[N,D])T=[D,M]
alpha_1_init = tf.assign(alpha_1, alpha_1_init_calculated) # [D,M]
H_1_init = h(mul(t(alpha_1_init), X_init)) # [M,N]
H_add = H_I + mul(H_1_init,t(H_1_init))
m = tf.assign(m,H_add)
H_pseudo_init = pseudo_inv(H_1_init,I_MxM,C) #[N,M]

beta_1_init_calculated = mul(Y_init, t(H_1_init))/sqr(tf.norm(H_1_init)) # [M,M]
beta_1_init_calculated = mul(Y_init,H_pseudo_init)

beta_1_init = tf.assign(beta_1, beta_1_init_calculated) # [M,M]
H_beta_1_init = mul(beta_1_init, t(mul(t(X_init), alpha_1_init))) # [M,N]
E_1_init = Y_init - H_beta_1_init # [M,N]

'''2nd subnet'''
#alpha_2_init_calculated = t(mul(h_(E_1_init), X_inv_init)) # [D,M]    
alpha_2_init_calculated = t(mul(asin(Y_init), X_inv_init)) # [D,M]

alpha_2_init = tf.assign(alpha_2, alpha_2_init_calculated) # [D,M]
H_2_init = h(mul(t(alpha_2_init), X_init)) # [M,N]
H_2_inv_init = pseudo_inv(H_2_init, I_MxM, C) # [M,N]
H_add = H_I + mul(H_2_init,t(H_2_init))
#m = tf.assign(m,H_add)
H_pseudo_init = pseudo_inv(H_2_init,I_MxM,C) #[N,M]

beta_2_init_calculated = mul(E_1_init, t(H_2_init))/sqr(tf.norm(H_2_init)) # [M,M]
beta_2_init_calculated = mul(E_1_init, H_pseudo_init)

beta_2_init = tf.assign(beta_2, beta_2_init_calculated) # [M,M]
H_beta_2_init = mul(beta_2_init, t(mul(t(X_init), alpha_2_init))) # [M,N]
E_2_init = Y_init - (H_beta_1_init+H_beta_2_init) # [M,N]

'''3rd subnetwork'''
alpha_3_init_calculated = t(mul(h_(E_2_init), X_inv_init)) # [D,M]    
alpha_3_init_calculated = t(mul(asin(Y_init), X_inv_init)) # [D,M]

alpha_3_init = tf.assign(alpha_3, alpha_3_init_calculated) # [D,M]
H_3_init = h(mul(t(alpha_3_init), X_init)) # [M,N]
H_3_inv_init = pseudo_inv(H_3_init, I_MxM, C) # [M,N]

beta_3_init_calculated = mul(E_2_init, t(H_3_init))/sqr(tf.norm(H_3_init)) # [M,M]
beta_3_init_calculated = mul(E_2_init, H_3_inv_init)

beta_3_init = tf.assign(beta_3, beta_3_init_calculated) # [M,M]
H_beta_3_init = mul(beta_3_init, t(mul(t(X_init), alpha_3_init))) # [M,N]
E_3_init = Y_init - (H_beta_3_init+H_beta_2_init+ H_beta_1_init) # [M,N]

#init_train_graph = H_beta_1_init
init_train_graph = E_3_init



In [12]:
beta_1_init

<tf.Tensor 'Assign_3:0' shape=(2, 2) dtype=float32_ref>

In [13]:
"""Evaluate initial-training graph"""
#logits_init =  subnet_output(alpha_1, beta_1, X_)
logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) + subnet_output(alpha_3, beta_3, X_)
loss_init = tf.losses.mean_squared_error(labels=Y_, predictions=logits_init)
accuracy_init = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_init, axis=1)), dtype=tf.float32))

In [14]:

'''With one subnetwork'''
logits_init = subnet_output(alpha_1, beta_1, X_) 
loss_init = tf.losses.mean_squared_error(labels=Y_, predictions=logits_init)
accuracy_init = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_init, axis=1)), dtype=tf.float32))

In [15]:
"""Initialize variables"""
init = tf.global_variables_initializer()
sess.run(init)

In [16]:
"""Initial training"""
sess.run(E_1_init, feed_dict={X_: x_train_init, Y: y_train_init})
print("Initial training done")

"""Initial training evaluation"""
tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_train, Y: y_train})
ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_test, Y: y_test})
print("Initial train training loss: ", tr_loss)
print("Initial train training accuracy: ", tr_acc)
print("Initial train testing loss: ", ts_loss)
print("Initial train testing accuracy: ", ts_acc)

Initial training done
Initial train training loss:  0.49712205
Initial train training accuracy:  0.5385568
Initial train testing loss:  0.49714962
Initial train testing accuracy:  0.52998036


In [17]:
"""Evaluate initial-training graph"""
#logits_init =  subnet_output(alpha_1, beta_1, X_)
logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) 
loss_init = tf.losses.mean_squared_error(labels=Y_, predictions=logits_init)
accuracy_init = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_init, axis=1)), dtype=tf.float32))

In [18]:
"""Initial training"""
import time
s = 0
t1 = time.time()
sess.run(E_2_init, feed_dict={X_: x_train_init, Y: y_train_init})
t2 = time.time()
s+=t2-t2
print("Initial training done")

"""Initial training evaluation"""
tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_train, Y: y_train})
ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_test, Y: y_test})
print("Initial train training loss: ", tr_loss)
print("Initial train training accuracy: ", tr_acc)
print("Initial train testing loss: ", ts_loss)
print("Initial train testing accuracy: ", ts_acc)

Initial training done
Initial train training loss:  0.49427783
Initial train training accuracy:  0.5385568
Initial train testing loss:  0.49433267
Initial train testing accuracy:  0.52998036


In [19]:
"""Evaluate initial-training graph"""
#logits_init =  subnet_output(alpha_1, beta_1, X_)
logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) + subnet_output(alpha_3, beta_3, X_)
loss_init = tf.losses.mean_squared_error(labels=Y_, predictions=logits_init)
accuracy_init = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_init, axis=1)), dtype=tf.float32))

In [20]:
"""Initial training"""
sess.run(init_train_graph, feed_dict={X_: x_train_init, Y: y_train_init})
print("Initial training done")

"""Initial training evaluation"""
tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_train, Y: y_train})
ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_test, Y: y_test})
print("Initial train training loss: ", tr_loss)
print("Initial train training accuracy: ", tr_acc)
print("Initial train testing loss: ", ts_loss)
print("Initial train testing accuracy: ", ts_acc)

Initial training done
Initial train training loss:  0.4914671
Initial train training accuracy:  0.5385568
Initial train testing loss:  0.4915487
Initial train testing accuracy:  0.52998036


In [21]:
logits_init = subnet_output(alpha_1, beta_1, X_)+ subnet_output(alpha_2, beta_2, X_)

In [22]:
logic = sess.run(logits_init, feed_dict={X_ : [x_test[4]]})
print(np.argmax(logic,axis =1))
print(y_test[4])
#plt.imshow(x_test[4200])

[1]
0


# Sequential Training

In [23]:
"""Sequential Training Graph"""
# D: input dimension
# N: number of input samples
# M: number of classes (number of outputs)
X_seq = t(X_) # [D,N]
Y_seq = t(Y_) # [M,N]
pseudo = mul(X_seq, X_) #DXD
k = tf.assign(k, tf.add(k,pseudo)) #DXD
k_inv = inv(k)

new = tf.matmul(tf.matmul(k_inv, X_seq),h_(Y_) - tf.matmul(X_, alpha_1))
alpha1_seq = tf.assign(alpha_1,tf.add(alpha_1,new)) #DXM
H_1_seq = h(mul(t(alpha1_seq), X_seq)) # [M,N]
m_su = mul(H_1_seq,t(H_1_seq))
m = tf.assign(m,tf.add(m,m_su))
m_inv = inv(m)
#update = tf.matmul(tf.matmul(m_inv,H_1_seq),h_(Y_seq)- tf.matmul())
H_pseudo_init = pseudo_inv(H_1_seq,I_MxM,C) #[N,M]
#UPDATE = tf.matmul(tf.matmul(K_inverse, HT), inverse_acti_y - tf.matmul(H, self.__outputWeight))
beta_1_seq_calculated = mul(Y_seq, H_pseudo_init) # [M,M]
beta_1_seq = tf.assign(beta_1, beta_1_seq_calculated) # [M,M]
H_beta_1_seq = mul(beta_1_seq, t(mul(X_, alpha1_seq))) # [M,N]
E_1_seq = Y_seq - H_beta_1_seq # [M,N]

'''2nd subnetwork'''

new = tf.matmul(tf.matmul(k_inv, X_seq),h_(Y_) - tf.matmul(X_, alpha_2))
alpha2_seq = tf.assign(alpha_2,tf.add(alpha_2,new)) #DXM
H_2_seq = h(mul(t(alpha2_seq), X_seq)) # [M,N]
H_pseudo_init = pseudo_inv(H_2_seq,I_MxM,C) #[N,M]
beta_2_seq_calculated = mul(E_1_seq, H_pseudo_init) # [M,M]
beta_2_seq = tf.assign(beta_2, beta_2_seq_calculated) # [M,M]
H_beta_2_seq = mul(beta_2_seq, t(mul(t(X_seq), alpha2_seq))) # [M,N]
E_2_seq = Y_seq - (H_beta_2_seq+ H_beta_1_seq) # [M,N]

'''3rd subnetwork'''
new = tf.matmul(tf.matmul(k_inv, X_seq),h_(Y_) - tf.matmul(X_, alpha_3))
alpha3_seq = tf.assign(alpha_3,tf.add(alpha_3,new)) #DXM
H_3_seq = h(mul(t(alpha3_seq), X_seq)) # [M,N]
H_pseudo_init = pseudo_inv(H_3_seq,I_MxM,C) #[N,M]
beta_3_seq_calculated = mul(E_2_seq, H_pseudo_init) # [M,M]
beta_3_seq = tf.assign(beta_3, beta_3_seq_calculated) # [M,M]
H_beta_3_seq = mul(beta_3_seq, t(mul(t(X_seq), alpha3_seq))) # [M,N]
E_3_seq = Y_seq - (H_beta_3_seq +H_beta_2_seq + H_beta_1_seq )# [M,N]
seq_train_graph = E_3_seq




In [24]:
"""Evaluate initial-training graph"""
logits_seq =  subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) + subnet_output(alpha_3, beta_3, X_)
#logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_)
loss_seq = tf.losses.mean_squared_error(labels=Y_, predictions=logits_seq)
accuracy_seq = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_seq, axis=1)), dtype=tf.float32))

In [26]:
"""Sequential training"""
batch_size = 1000

epoch_train_accuracy = []
epoch_test_accuracy = []
t1 = time.time()
for epoch in range(205):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        sess.run(seq_train_graph, feed_dict={X_: x_batch, Y: y_batch})
        #pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, train_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_train, Y: y_train})
    [test_loss, test_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_test, Y: y_test})
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)
#sess.run(init_train_graph, feed_dict={X: x_train_init, Y: y_train_init})
t2 = time.time()
s+=(t2-t1)/205
print("Sequential training done")

"""Sequential training evaluation"""
#tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X: x_train, Y: y_train})
#ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X: x_test, Y: y_test})
#print("Sequential train training loss: ", tr_loss)
#print("Sequential train training accuracy: ", tr_acc)
#print("Sequential train testing loss: ", ts_loss)
#print("Sequential train testing accuracy: ", ts_acc)

train_loss: 0.192263, train_accuracy: 0.735731
test_loss: 0.189003, test_accuracy: 0.746715
train_loss: 0.192259, train_accuracy: 0.735619
test_loss: 0.189000, test_accuracy: 0.746715
train_loss: 0.192255, train_accuracy: 0.735619
test_loss: 0.188998, test_accuracy: 0.746866
train_loss: 0.192250, train_accuracy: 0.735675
test_loss: 0.188996, test_accuracy: 0.746866
train_loss: 0.192246, train_accuracy: 0.735619
test_loss: 0.188993, test_accuracy: 0.747168
train_loss: 0.192241, train_accuracy: 0.735787
test_loss: 0.188991, test_accuracy: 0.747017
train_loss: 0.192237, train_accuracy: 0.735844
test_loss: 0.188988, test_accuracy: 0.746866
train_loss: 0.192232, train_accuracy: 0.735787
test_loss: 0.188986, test_accuracy: 0.746866
train_loss: 0.192228, train_accuracy: 0.735844
test_loss: 0.188984, test_accuracy: 0.747017
train_loss: 0.192224, train_accuracy: 0.735844
test_loss: 0.188981, test_accuracy: 0.747017
train_loss: 0.192219, train_accuracy: 0.735731
test_loss: 0.188979, test_accurac

train_loss: 0.191973, train_accuracy: 0.738095
test_loss: 0.188842, test_accuracy: 0.749887
train_loss: 0.191971, train_accuracy: 0.738095
test_loss: 0.188841, test_accuracy: 0.750189
train_loss: 0.191969, train_accuracy: 0.738208
test_loss: 0.188840, test_accuracy: 0.750491
train_loss: 0.191967, train_accuracy: 0.738208
test_loss: 0.188838, test_accuracy: 0.750642
train_loss: 0.191965, train_accuracy: 0.738152
test_loss: 0.188837, test_accuracy: 0.750793
train_loss: 0.191963, train_accuracy: 0.738208
test_loss: 0.188836, test_accuracy: 0.750793
train_loss: 0.191961, train_accuracy: 0.738208
test_loss: 0.188835, test_accuracy: 0.750944
train_loss: 0.191959, train_accuracy: 0.738208
test_loss: 0.188834, test_accuracy: 0.750944
train_loss: 0.191957, train_accuracy: 0.738264
test_loss: 0.188833, test_accuracy: 0.751246
train_loss: 0.191955, train_accuracy: 0.738264
test_loss: 0.188832, test_accuracy: 0.751095
train_loss: 0.191953, train_accuracy: 0.738208
test_loss: 0.188831, test_accurac

train_loss: 0.191839, train_accuracy: 0.739615
test_loss: 0.188774, test_accuracy: 0.752001
train_loss: 0.191838, train_accuracy: 0.739615
test_loss: 0.188773, test_accuracy: 0.752303
train_loss: 0.191837, train_accuracy: 0.739559
test_loss: 0.188773, test_accuracy: 0.752454
train_loss: 0.191836, train_accuracy: 0.739502
test_loss: 0.188772, test_accuracy: 0.752454
train_loss: 0.191835, train_accuracy: 0.739446
test_loss: 0.188772, test_accuracy: 0.752454
train_loss: 0.191834, train_accuracy: 0.739559
test_loss: 0.188771, test_accuracy: 0.752454
train_loss: 0.191833, train_accuracy: 0.739559
test_loss: 0.188771, test_accuracy: 0.752454
train_loss: 0.191832, train_accuracy: 0.739615
test_loss: 0.188771, test_accuracy: 0.752454
train_loss: 0.191832, train_accuracy: 0.739615
test_loss: 0.188770, test_accuracy: 0.752454
train_loss: 0.191831, train_accuracy: 0.739671
test_loss: 0.188770, test_accuracy: 0.752303
train_loss: 0.191830, train_accuracy: 0.739728
test_loss: 0.188769, test_accurac

'Sequential training evaluation'

In [28]:
s

0.11988424557011301

In [24]:
logits_seq =  subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) 
#logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_)
loss_seq = tf.losses.mean_squared_error(labels=Y_, predictions=logits_seq)
accuracy_seq = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_seq, axis=1)), dtype=tf.float32))

"""Initialize variables"""
init = tf.global_variables_initializer()
sess.run(init)

In [25]:
"""Sequential training"""
batch_size = 250

epoch_train_accuracy = []
epoch_test_accuracy = []
for epoch in range(700):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        sess.run(E_2_seq, feed_dict={X_: x_batch, Y: y_batch})
        #pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, train_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_train, Y: y_train})
    [test_loss, test_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_test, Y: y_test})
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)
#sess.run(init_train_graph, feed_dict={X: x_train_init, Y: y_train_init})
print("Sequential training done")

"""Sequential training evaluation"""

train_loss: 0.145569, train_accuracy: 0.684512
test_loss: 0.152528, test_accuracy: 0.661046
train_loss: 0.142330, train_accuracy: 0.719885
test_loss: 0.149371, test_accuracy: 0.687184
train_loss: 0.139225, train_accuracy: 0.748566
test_loss: 0.146340, test_accuracy: 0.713322
train_loss: 0.136255, train_accuracy: 0.778203
test_loss: 0.143435, test_accuracy: 0.737774
train_loss: 0.133418, train_accuracy: 0.810707
test_loss: 0.140656, test_accuracy: 0.761383
train_loss: 0.130713, train_accuracy: 0.828872
test_loss: 0.138003, test_accuracy: 0.779089
train_loss: 0.128138, train_accuracy: 0.845124
test_loss: 0.135472, test_accuracy: 0.794266
train_loss: 0.125690, train_accuracy: 0.855641
test_loss: 0.133062, test_accuracy: 0.806914
train_loss: 0.123365, train_accuracy: 0.868069
test_loss: 0.130770, test_accuracy: 0.818718
train_loss: 0.121160, train_accuracy: 0.879541
test_loss: 0.128592, test_accuracy: 0.829680
train_loss: 0.119071, train_accuracy: 0.884321
test_loss: 0.126525, test_accurac

train_loss: 0.085969, train_accuracy: 0.936902
test_loss: 0.092838, test_accuracy: 0.907251
train_loss: 0.085992, train_accuracy: 0.936902
test_loss: 0.092858, test_accuracy: 0.907251
train_loss: 0.086016, train_accuracy: 0.936902
test_loss: 0.092880, test_accuracy: 0.907251
train_loss: 0.086041, train_accuracy: 0.936902
test_loss: 0.092903, test_accuracy: 0.907251
train_loss: 0.086069, train_accuracy: 0.936902
test_loss: 0.092928, test_accuracy: 0.908094
train_loss: 0.086097, train_accuracy: 0.936902
test_loss: 0.092954, test_accuracy: 0.908094
train_loss: 0.086127, train_accuracy: 0.936902
test_loss: 0.092982, test_accuracy: 0.908094
train_loss: 0.086158, train_accuracy: 0.936902
test_loss: 0.093011, test_accuracy: 0.908094
train_loss: 0.086191, train_accuracy: 0.936902
test_loss: 0.093042, test_accuracy: 0.908094
train_loss: 0.086224, train_accuracy: 0.936902
test_loss: 0.093073, test_accuracy: 0.908094
train_loss: 0.086259, train_accuracy: 0.936902
test_loss: 0.093106, test_accurac

train_loss: 0.090915, train_accuracy: 0.943595
test_loss: 0.097673, test_accuracy: 0.908094
train_loss: 0.090975, train_accuracy: 0.943595
test_loss: 0.097732, test_accuracy: 0.908094
train_loss: 0.091034, train_accuracy: 0.944551
test_loss: 0.097791, test_accuracy: 0.908094
train_loss: 0.091093, train_accuracy: 0.944551
test_loss: 0.097849, test_accuracy: 0.908938
train_loss: 0.091151, train_accuracy: 0.944551
test_loss: 0.097908, test_accuracy: 0.908938
train_loss: 0.091210, train_accuracy: 0.944551
test_loss: 0.097966, test_accuracy: 0.908094
train_loss: 0.091269, train_accuracy: 0.944551
test_loss: 0.098025, test_accuracy: 0.908094
train_loss: 0.091328, train_accuracy: 0.946463
test_loss: 0.098083, test_accuracy: 0.908094
train_loss: 0.091386, train_accuracy: 0.946463
test_loss: 0.098141, test_accuracy: 0.908094
train_loss: 0.091444, train_accuracy: 0.946463
test_loss: 0.098199, test_accuracy: 0.908094
train_loss: 0.091503, train_accuracy: 0.946463
test_loss: 0.098257, test_accurac

train_loss: 0.095909, train_accuracy: 0.950287
test_loss: 0.102723, test_accuracy: 0.912310
train_loss: 0.095953, train_accuracy: 0.950287
test_loss: 0.102768, test_accuracy: 0.912310
train_loss: 0.095996, train_accuracy: 0.950287
test_loss: 0.102813, test_accuracy: 0.911467
train_loss: 0.096038, train_accuracy: 0.950287
test_loss: 0.102857, test_accuracy: 0.911467
train_loss: 0.096081, train_accuracy: 0.950287
test_loss: 0.102902, test_accuracy: 0.911467
train_loss: 0.096123, train_accuracy: 0.950287
test_loss: 0.102946, test_accuracy: 0.911467
train_loss: 0.096165, train_accuracy: 0.950287
test_loss: 0.102990, test_accuracy: 0.911467
train_loss: 0.096207, train_accuracy: 0.950287
test_loss: 0.103034, test_accuracy: 0.911467
train_loss: 0.096249, train_accuracy: 0.950287
test_loss: 0.103077, test_accuracy: 0.912310
train_loss: 0.096291, train_accuracy: 0.950287
test_loss: 0.103121, test_accuracy: 0.912310
train_loss: 0.096332, train_accuracy: 0.950287
test_loss: 0.103164, test_accurac

train_loss: 0.099229, train_accuracy: 0.955067
test_loss: 0.106301, test_accuracy: 0.913997
train_loss: 0.099256, train_accuracy: 0.954111
test_loss: 0.106331, test_accuracy: 0.913997
train_loss: 0.099282, train_accuracy: 0.954111
test_loss: 0.106361, test_accuracy: 0.913997
train_loss: 0.099308, train_accuracy: 0.954111
test_loss: 0.106391, test_accuracy: 0.913997
train_loss: 0.099335, train_accuracy: 0.954111
test_loss: 0.106421, test_accuracy: 0.913997
train_loss: 0.099361, train_accuracy: 0.954111
test_loss: 0.106451, test_accuracy: 0.914840
train_loss: 0.099387, train_accuracy: 0.954111
test_loss: 0.106480, test_accuracy: 0.914840
train_loss: 0.099412, train_accuracy: 0.954111
test_loss: 0.106509, test_accuracy: 0.914840
train_loss: 0.099438, train_accuracy: 0.954111
test_loss: 0.106539, test_accuracy: 0.914840
train_loss: 0.099463, train_accuracy: 0.954111
test_loss: 0.106568, test_accuracy: 0.914840
train_loss: 0.099489, train_accuracy: 0.954111
test_loss: 0.106597, test_accurac

train_loss: 0.101211, train_accuracy: 0.955067
test_loss: 0.108675, test_accuracy: 0.912310
train_loss: 0.101226, train_accuracy: 0.955067
test_loss: 0.108694, test_accuracy: 0.912310
train_loss: 0.101242, train_accuracy: 0.955067
test_loss: 0.108714, test_accuracy: 0.912310
train_loss: 0.101257, train_accuracy: 0.955067
test_loss: 0.108734, test_accuracy: 0.912310
train_loss: 0.101272, train_accuracy: 0.955067
test_loss: 0.108754, test_accuracy: 0.912310
train_loss: 0.101287, train_accuracy: 0.955067
test_loss: 0.108773, test_accuracy: 0.912310
train_loss: 0.101302, train_accuracy: 0.955067
test_loss: 0.108793, test_accuracy: 0.912310
train_loss: 0.101317, train_accuracy: 0.955067
test_loss: 0.108812, test_accuracy: 0.912310
train_loss: 0.101332, train_accuracy: 0.955067
test_loss: 0.108832, test_accuracy: 0.912310
train_loss: 0.101347, train_accuracy: 0.955067
test_loss: 0.108851, test_accuracy: 0.912310
train_loss: 0.101361, train_accuracy: 0.955067
test_loss: 0.108870, test_accurac

train_loss: 0.102407, train_accuracy: 0.956979
test_loss: 0.110351, test_accuracy: 0.909781
train_loss: 0.102416, train_accuracy: 0.956979
test_loss: 0.110364, test_accuracy: 0.909781
train_loss: 0.102424, train_accuracy: 0.956023
test_loss: 0.110377, test_accuracy: 0.909781
train_loss: 0.102432, train_accuracy: 0.956023
test_loss: 0.110390, test_accuracy: 0.909781
train_loss: 0.102440, train_accuracy: 0.956023
test_loss: 0.110403, test_accuracy: 0.909781
train_loss: 0.102449, train_accuracy: 0.956023
test_loss: 0.110416, test_accuracy: 0.909781
train_loss: 0.102457, train_accuracy: 0.956023
test_loss: 0.110429, test_accuracy: 0.909781
train_loss: 0.102465, train_accuracy: 0.956979
test_loss: 0.110442, test_accuracy: 0.909781
train_loss: 0.102473, train_accuracy: 0.956979
test_loss: 0.110455, test_accuracy: 0.909781
train_loss: 0.102481, train_accuracy: 0.956979
test_loss: 0.110468, test_accuracy: 0.909781
train_loss: 0.102489, train_accuracy: 0.956979
test_loss: 0.110480, test_accurac

train_loss: 0.103071, train_accuracy: 0.960803
test_loss: 0.111511, test_accuracy: 0.915683
train_loss: 0.103075, train_accuracy: 0.960803
test_loss: 0.111520, test_accuracy: 0.915683
train_loss: 0.103080, train_accuracy: 0.960803
test_loss: 0.111529, test_accuracy: 0.915683
train_loss: 0.103084, train_accuracy: 0.960803
test_loss: 0.111538, test_accuracy: 0.915683
train_loss: 0.103089, train_accuracy: 0.960803
test_loss: 0.111547, test_accuracy: 0.915683
train_loss: 0.103093, train_accuracy: 0.959847
test_loss: 0.111556, test_accuracy: 0.915683
train_loss: 0.103097, train_accuracy: 0.959847
test_loss: 0.111565, test_accuracy: 0.915683
train_loss: 0.103102, train_accuracy: 0.959847
test_loss: 0.111574, test_accuracy: 0.915683
train_loss: 0.103106, train_accuracy: 0.959847
test_loss: 0.111583, test_accuracy: 0.915683
train_loss: 0.103110, train_accuracy: 0.959847
test_loss: 0.111592, test_accuracy: 0.915683
train_loss: 0.103115, train_accuracy: 0.959847
test_loss: 0.111601, test_accurac

'Sequential training evaluation'

In [ ]:
"""Evaluate initial-training graph"""
logits_seq =  subnet_output(alpha_1, beta_1, X_) 
#logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_)
loss_seq = tf.losses.mean_squared_error(labels=Y_, predictions=logits_seq)
accuracy_seq = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_seq, axis=1)), dtype=tf.float32))

In [ ]:
"""Initialize variables"""
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
"""Sequential training"""
batch_size = 250

epoch_train_accuracy = []
epoch_test_accuracy = []
for epoch in range(40):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        sess.run(E_1_seq, feed_dict={X_: x_batch, Y: y_batch})
        #pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, train_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_train, Y: y_train})
    [test_loss, test_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_test, Y: y_test})
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)
#sess.run(init_train_graph, feed_dict={X: x_train_init, Y: y_train_init})
print("Sequential training done")

"""Sequential training evaluation"""

In [ ]:
logits_ = subnet_output(alpha_1, beta_1, X_) 
logits__ = sess.run(logits_, feed_dict={X: [x_test[4000]]})
print(logits__)
print(np.argmax(logits__))
print(y_test[4000])
plt.imshow(x_test[4000])

In [ ]:
def visualize_alpha(alpha, size):
    tmp = sess.run(alpha)
    plt.figure(figsize=(10,4))
    for i in range(2):
        for j in range(5):
            plt.subplot(2,5,i*5+j+1)
            plt.imshow(np.reshape(tmp[:,i*5+j], [size,size]))

def visualize_beta(beta):
    tmp = sess.run(beta)
    plt.figure(figsize=(4,4))
    plt.imshow(tmp)
    
            
"""visualize subnet nodes"""            
visualize_alpha(alpha_1, 10)
visualize_beta(beta_1)
visualize_alpha(alpha_2, 10)
visualize_beta(beta_2)

In [ ]:
from ml_elm import ML_ELM

mlelm1 = ML_ELM(input_size=input_size, output_size=output_size, name='mlelm1')


In [ ]:
sess.run(tf.math.sin(tf.constant([0.9])))